In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [6]:
code_dir = os.getcwd()
base_dir = code_dir.replace("/Z_codes_local", "")

In [11]:
def mouse_to_human(inList, reverse=False):
    inList = [str(x).upper() for x in inList]
    
    mmhs_ref = base_dir + "/Human_to_Mouse_geneNames.csv"
    mmhs_df = pd.read_csv(mmhs_ref)
    
    
    if not reverse:
        old_names = mmhs_df['mouse_geneName'].values.tolist()
        new_names = mmhs_df['human_geneName'].values.tolist()
    else:
        old_names = mmhs_df['human_geneName'].values.tolist()
        new_names = mmhs_df['mouse_geneName'].values.tolist()
    old_names = [ str(x).upper() for x in old_names]
    new_names = [ str(x).upper() for x in new_names]
    
    outList = []
    for i in inList:
        if i in old_names:
            idx = old_names.index(i)
            outList.append(new_names[idx])
        else:
            outList.append("")
    if reverse:
        outList = [x.title() for x in outList]
        return(outList)
    else:
        return(outList)

## 0. Convert mouse references to human

In [12]:
input_dir = base_dir + '/X_Formatted_csv/*'

mouse_sig_files = glob.glob("%s/*mouse.csv"%input_dir)
for i in mouse_sig_files:
    i_human = i.replace("_mouse.csv", ".csv")
    if not os.path.exists(i_human):
        print(i)
        i_df = pd.read_csv(i)
        i_df['gene_symbol'] = mouse_to_human(i_df['gene_symbol'].values.tolist())
        i_df = i_df[i_df['gene_symbol'] != '']
        i_df.to_csv(i_human, index=False)

/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/202009_CD8/2018_IMMUNITY_Wang_signatures_mouse.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/202009_CD8/2016_SCIENCE_Mackay_signatures_mouse.csv


## 1. Summarize all references 

In [13]:
wk_dir = base_dir + "/X_GeneSignatures_hs"
os.chdir(wk_dir)

input_dir = base_dir + '/X_Formatted_csv'

all_sig_files = glob.glob("%s/*/*signatures.csv"%input_dir)
all_sig_df = pd.DataFrame()

for i in all_sig_files:
    i_basename = i.split("/")[-1].replace("_signatures.csv", "")
    i_df = pd.read_csv(i)[['gs_name', 'gene_symbol']]
    i_df['gs_name'] = [i_basename + "---" + x for x in i_df['gs_name'].values]
    all_sig_df = all_sig_df.append(i_df)

all_sig_df['gs_name'] = [x.replace(" ", "_").replace("/","-").replace("ï", "i") for x in all_sig_df['gs_name'].values]
all_sig_df_sum = all_sig_df.groupby('gs_name').count()
all_sig_df_sum.columns = ['gene_number']

In [14]:
all_sig_df.to_csv("all_human_T_cell_signatures.csv", index=False)
all_sig_df_sum.to_csv("all_human_T_cell_gs_signatures_summary.csv")

## 2. Create subset references

In [15]:
anno_file_dir = base_dir + '/Y_annotated'
anno_files = glob.glob("%s/plotuse_gs*.csv"%anno_file_dir)
anno_files.sort()

old_anno_file = anno_files[-1]
old_date = old_anno_file.split("/")[-1].replace(".csv", "").replace("plotuse_gs_", "").replace("_anno","")

#--- Use the annotated file if it is available
if  os.path.exists(old_anno_file.replace("plotuse", "anno_plotuse")):
    old_anno_df = pd.read_csv(old_anno_file.replace("plotuse", "anno_plotuse"))
else:
    old_anno_df = pd.read_csv(old_anno_file)

In [16]:
wk_dir = base_dir + "/X_GeneSignatures_hs"
os.chdir(wk_dir)

all_sig_df = pd.read_csv("all_human_T_cell_signatures.csv")
gs_anno = old_anno_df

In [8]:
gs_anno_use_types = np.unique(gs_anno_use['type'])

In [9]:
for gs_type in gs_anno_use_types:
    gs_type_names = gs_anno_use[gs_anno_use['type'] == gs_type]['gs_name'].values
    all_sig_df_gs_type = all_sig_df_use[[True if x in gs_type_names else False for x in all_sig_df_use['gs_name'].values]]
    all_sig_df_gs_type.to_csv("%s_hs_sigs.csv"%gs_type, index=False)